In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
import scipy
import pandas as pd
import os


Global Parameter: Angle of the Cantilever and Poisson's Ratio

In [2]:
theta=18 # Cantilever angle 
nu=0.5 # Poisson'S Ratio



In [3]:
plt.style.use('ggplot')

In [4]:
Path ='/home/bjoern/Desktop/Nano_Rheology/cell_data/'
files=os.listdir(Path)
fil=files[1]


create DataFrame from csv file

In [5]:
def dataFr(Path, file):
    df=pd.read_csv(Path+file)
    df.drop(u'Unnamed: 0',1,inplace=True)
    return df

extract parameters: date, cellID, spring constant, cell size, and index of the measurement. Insert parameters into dictionary

In [6]:
def df_para2dic(dic, df):    
    
    dic['date']= df['Date'][0]
    dic['cell_ID']= df['cellID'][0]
    dic['spring_C'] = df['springConstant'][0]
    dic['cell_Size'] = df['size'][0]
    dic['index'] = df['index'][0]
    return dic
    #global_para = [date, cellID, index, springC, cellSize]
    

fit parameters into dictionary

In [7]:
def fit_para2dic(dic, popt, pcov):   
    
    dic['E']= popt[0]
    dic['x0']= popt[1]
    dic['F0']= popt[2] 

    dic['var_E']= pcov[0][0]
    dic['var_x0']= pcov[1][1]
    dic['var_F0']= pcov[2][2]
    
    return dic




Herz-Sneddon Model 

In [ ]:
def sneddon(x, E, theta, nu, x0, F0):
    tanTheta=np.tan(theta*np.pi/180)
    return (x-x0)**2*(E*2*tanTheta)/((1-nu**2)*np.pi)+F0  

In [ ]:
def fit_sneddon(df): 
   
    extend = df[df['modus'] == 'extend']
    
    x = -extend['measuredHeight'].values
    y = extend['vDeflection'].values
    
    x_red = x[x>-0.1e-7]
    y_red = y[x>-0.1e-7]

    #boundary=([-10e3,-np.inf,-np.inf],[10e6, np.inf, np.inf])# does not work

    popt,pcov=curve_fit(lambda x_red, E, x0, F0: sneddon(x_red, E, theta, nu, x0, F0),
                    x_red, y_red, p0=[10e6, 0, 0])#, bounds=boundary)
    return (popt,pcov)

plot correlation between estimated and measured force curve

In [ ]:
def plot_cor(pos, rep):
    
    if pos % rep == 0:
        
        y_sneddon=sneddon(x_red, popt[0], theta, nu, popt[1], popt[2])
        
        plt.figure()
        #plot parameter
        plt.title('fit correlation')
        plt.xlabel('measured Force, N')
        plt.ylabel('fit F - measured F, N')
        plt.savefig('fit_correlation.pdf')


        plt.plot(y_red, y_sneddon-y_red, '.')
        plt.plot(y_red, [0]*len(y_red))
    

In [8]:
def plot_osci(t, f, h):
    plt.figure()
    
    plt.subplot(1,2,1)
    plt.plot(t,f-f.mean())
    plt.xlabel('time,s')
    plt.title('F(t)')

    plt.subplot(1,2,2)
    plt.plot(t,h-h.mean())
    plt.xlabel('time,s')
    plt.title('h(t)')


In [9]:
def plot_psd(N, freq,f_fft_abs, f_max, h_fft_abs, h_max, xlim, ylim): # N, freq,f_fft_abs, f_max, h_fft_abs, h_max, xlim
    
    plt.figure(1)
    fig, ax = plt.subplots()
    
    ax.plot(freq[0:N/2],f_fft_abs[0:N/2]/f_max, label='force')
    ax.plot(freq[0:N/2],h_fft_abs[0:N/2]/h_max, label='height')
    ax.legend()
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax.set_xlabel('frequency, Hz')
    ax.set_ylabel('Absolute Values - Normalized')


In [41]:
def G_star(df):
    
    dic={}
    df=df[df['modus'] == 'modulation'] # reduced dataframe (only modulation segments)

    frequencies = df['frequency'].unique()
    frequencies = frequencies[~np.isnan(frequencies)] #list of used frequencies
    #print(frequencies)

    df_mod=df[df['frequency'] == frequencies[1]] # df of one modulation segment with one frequency

    for pos, fre in enumerate(frequencies):

        df_mod=df[df['frequency'] == fre]

        sf=df_mod['sampleFrequency'].values[pos]
        t=df_mod['time']
        f=df_mod['vDeflection']
        h=df_mod['measuredHeight']
        fm=f.mean()
        hm=h.mean()


        #plot oscillation
        #plot_osci(t, f, h)

        #correct for offset force
        f_cor = f - f.mean()
        #correct for offset height
        indentation_depth = h.mean()
        h_cor = h - indentation_depth

        #sample size
        N = len(f_cor) 

        # fast fouriertransform
        f_fft=np.fft.fft(f_cor)
        h_fft=np.fft.fft(h_cor)
        # absolute values
        f_fft_abs=np.abs(f_fft)
        h_fft_abs=np.abs(h_fft)

        #frequency vector
        freq=np.fft.fftfreq(t.shape[0], d=1/sf)

        #max h and f in the fourier transform
        f_max = max(f_fft_abs[0:N / 2])
        h_max = max(h_fft_abs[0:N / 2])    

         #maxfreq         
        maxfreq_f=freq[f_fft_abs == f_max]# negative and positive
        maxfreq_h=freq[h_fft_abs == h_max]
        #print(maxfreq_f[0])
        #print(maxfreq_h[0])

        #plot power spectral density
        ylim=(-0.1, 1.1)
        xlim=(0, 120)
        #plot_psd(N, freq,f_fft_abs, f_max, h_fft_abs, h_max, xlim, ylim)

        F_f=f_fft[f_fft_abs == max(f_fft_abs)]# F(freq)
        if ~(fre*0.9 < maxfreq_f[0] < fre*1.1) :
            print('calculated frequency is not in range. used freq:' + str(fre) + 'calulated freq:'+ str(maxfreq_f[0]))

        # F(f) and h(f)
        f_fft_red=f_fft[0:N / 2]
        F=f_fft_red[f_fft_abs[0:N / 2] == f_max]
        h_fft_red=h_fft[0:N / 2]
        h=h_fft_red[h_fft_abs[0:N / 2] == h_max]        

        #dic[fre]=h/F
        Ha=h/F
        b0=0
        delta0=indentation_depth
        
        G=(1 - nu) * np.pi * (Ha - (j * fm * b0)) / (delta0 * np.tan(theta) * 8);
        
        dic[fre]=G

    #print(dic)
    
    #dic[1][0]
    return dic

In [22]:
df = dataFr(Path, fil)
df1 = dataFr(Path, fil)
files[0:3]
di1={1:'E',2:'r'}
di2={3:'E',4:'r'}
di1.update(di2)
di1
#print(df.head(1))

{1: 'E', 2: 'r', 3: 'E', 4: 'r'}

#### create dataframe from all dictionaries of all files 

In [36]:
ratio_F_h_list=[]
theta=18
nu=0.5

for pos, file in enumerate(files):
    
    df = dataFr(Path, file)
    dic=f_fft_h_fft(df)
    df_para2dic(dic, df)
    
    dic['theta'] = theta
    dic['nu'] = nu
    #print(dic)
    ratio_F_h_list.append(dic)
    #ratio_F_h_list
    
df_fh=pd.DataFrame(ratio_F_h_list)
df_fh

/usr/lib/python3/dist-packages/ipykernel/__main__.py:47: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/lib/python3/dist-packages/ipykernel/__main__.py:48: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/lib/python3/dist-packages/ipykernel/__main__.py:66: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/lib/python3/dist-packages/ipykernel/__main__.py:67: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/lib/python3/dist-packages/ipykernel/__main__.py:68: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/lib/python3/dist-packages/ipykernel/__main__.py:69: VisibleDeprecationWarning: using a non-integer number instead of an integer will result 

calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not in range. used freq:1.0calulated freq:0.333333333333
calculated frequency is not 

,1.0,5.0,50.0,100.0,index,spring_C,10.0,cell_ID,date,cell_Size
0,[(-75.8019019175+8.42446034808j)],[(-70.4330606217+11.2563697461j)],[(-48.4949403136+21.6356935713j)],[(-45.5580223951+25.9645785257j)],38,0.264026,[(-67.3843979948+10.6542509686j)],23,Tue Nov 01 15:09:13 CET 2016,5
1,[(-24.8425255806+1.56971980729j)],[(-22.2921742534+1.62895925892j)],[(-20.180013039+1.77603299975j)],[(-19.0433813503+3.70885193149j)],73,0.275080,[(-22.0896743838+2.25959120301j)],29,Wed Oct 26 14:52:59 CEST 2016,3
2,[(-11.2701528781+0.234177079717j)],[(-10.8984651388+0.248720548667j)],[(-10.5636633176+0.565357837657j)],[(-10.0626907979+1.03510213952j)],21,0.262393,[(-10.7716947626+0.455943165945j)],17,Wed Nov 02 13:12:25 CET 2016,5
3,[(-23.0881658776+0.527601405645j)],[(-20.7828349015+0.266418441707j)],[(-21.1603234806+1.52735742236j)],[(-19.6615073401+2.19066283145j)],31,0.264026,[(-20.713124933+0.60981688538j)],23,Tue Nov 01 15:19:34 CET 2016,5
4,[(-30.957699399+1.55905120945j)],[(-29.3728419885+1.36614431712j)],[(-27.385204596+2.77988983071j)],[(-26.2814318655+5.85069275653j)],99,0.275049,[(-28.0663583891+1.36045797542j)],10,Tue Nov 08 13:34:45 CET 2016,5
5,[(-13.9087029657+0.0928567020769j)],[(-13.7401000346+0.153492776266j)],[(-13.6286149645+0.636174947877j)],[(-13.3126987102+1.48898955816j)],16,0.275049,[(-13.718152547+0.110667906906j)],4,Tue Nov 08 14:46:49 CET 2016,3
6,[(-17.2897363469+0.0284468700762j)],[(-16.6585298957+0.184066995819j)],[(-16.5455175337+0.759722246079j)],[(-16.3028729703+1.40204443735j)],83,0.275049,[(-16.51300356+0.279005258125j)],4,Tue Nov 08 15:03:19 CET 2016,3
7,[(-11.8438889145-0.0269910511271j)],[(-11.5449250472+0.0283297353746j)],[(-11.2904692295+0.43091953599j)],[(-10.9854163667+0.946103572463j)],40,0.275049,[(-11.3131976282+0.17977448144j)],10,Tue Nov 08 13:07:32 CET 2016,5
8,[(-21.8420618138+1.07834547225j)],[(-19.8147235833+0.486008466256j)],[(-18.3032359175+1.75960455203j)],[(-17.5865413308+2.5852694893j)],56,0.275049,[(-19.1935558785+1.36046240349j)],10,Tue Nov 08 13:40:23 CET 2016,5
9,[(-14.2792796527-0.0800035253703j)],[(-13.713686136+0.0854764445009j)],[(-13.3389525811+0.689794523195j)],[(-13.3972007528+1.28992361766j)],36,0.262393,[(-13.2303592859+0.316998962793j)],14,Wed Nov 02 15:40:43 CET 2016,3


### save df_fh.p file of the created dataframe df!!

In [31]:
import pickle
with open('df_fh.p', 'wb') as f:
    
    pickle.dump(df_fh, f)
    

### load df_fh.p file of the created dataframe df

In [ ]:
import pickle
with open('df_fh.p','rb') as f:
    df_fh=pickle.load(f)

In [40]:
df_fh.head()



,1.0,5.0,50.0,100.0,index,spring_C,10.0,cell_ID,date,cell_Size
0,[(-75.8019019175+8.42446034808j)],[(-70.4330606217+11.2563697461j)],[(-48.4949403136+21.6356935713j)],[(-45.5580223951+25.9645785257j)],38,0.264026,[(-67.3843979948+10.6542509686j)],23,Tue Nov 01 15:09:13 CET 2016,5
1,[(-24.8425255806+1.56971980729j)],[(-22.2921742534+1.62895925892j)],[(-20.180013039+1.77603299975j)],[(-19.0433813503+3.70885193149j)],73,0.275080,[(-22.0896743838+2.25959120301j)],29,Wed Oct 26 14:52:59 CEST 2016,3
2,[(-11.2701528781+0.234177079717j)],[(-10.8984651388+0.248720548667j)],[(-10.5636633176+0.565357837657j)],[(-10.0626907979+1.03510213952j)],21,0.262393,[(-10.7716947626+0.455943165945j)],17,Wed Nov 02 13:12:25 CET 2016,5
3,[(-23.0881658776+0.527601405645j)],[(-20.7828349015+0.266418441707j)],[(-21.1603234806+1.52735742236j)],[(-19.6615073401+2.19066283145j)],31,0.264026,[(-20.713124933+0.60981688538j)],23,Tue Nov 01 15:19:34 CET 2016,5
4,[(-30.957699399+1.55905120945j)],[(-29.3728419885+1.36614431712j)],[(-27.385204596+2.77988983071j)],[(-26.2814318655+5.85069275653j)],99,0.275049,[(-28.0663583891+1.36045797542j)],10,Tue Nov 08 13:34:45 CET 2016,5


In [ ]:

data=[]
error_count=0

for pos, file in enumerate(files[0:1]):
    
    dic={}
    dic['theta'] = theta
    dic['nu'] = nu
    
    df = dataFr(Path, file) # create suitable data frame
    dic = df_para2dic(dic, df)  #extract global parameters from df
    
   
   
    modulation = df[df['modus'] == 'modulation']
    
    
    x = -extend['measuredHeight'].values
    y = extend['vDeflection'].values
    
    x_red = x[x>-0.1e-7]
    y_red = y[x>-0.1e-7]

    #boundary=([-10e3,-np.inf,-np.inf],[10e6, np.inf, np.inf])# does not work
    
    try:
        popt, pcov=curve_fit(lambda x_red, E, x0, F0: sneddon(x_red, E, theta, nu, x0, F0),
                    x_red, y_red, p0=[10e6, 0, 0])#, bounds=boundary)

    except RuntimeError:
        print("Error - curve_fit failed")
        error_count+=1
    except TypeError:
        print("Improper input:")
        error_count+=1
    
    
    #popt, pcov=curve_fit(lambda x_red, E, x0, F0: sneddon(x_red, E, theta, nu, x0, F0),
      #              x_red, y_red, p0=[10e6, 0, 0])#, bounds=boundary)
    
    
    
    plot_cor(pos, 4)
    
    
    
    
    dic = fit_para2dic(dic, popt, pcov)
    
    dic2list=[dic]
    
    data.extend(dic2list)

    

    
df=pd.DataFrame(data)
df.head(10)

    


In [ ]:
print(df.columns)
#for pos,size in enumerate(df['cell_Size'].unique()):
df_3um=df[df['cell_Size']==3]
df_4um=df[df['cell_Size']==4]
df_5um=df[df['cell_Size']==5]
len(df_5um)

df_3um.groupby('cell_ID').describe()

#for pos,size in enumerate(df['cell_Size'].unique()) df_3um[]

In [ ]:
df2=df.groupby(['cell_Size','cell_ID'])['E'].mean()*1e-6

df3=df2.unstack(level=0)#.plot(kind='box')
#print(df3)
pd.melt(df3).plot(kind='scatter',x='cell_Size', y='value')
df3.mean()


In [ ]:
cell_ID_small=dfmE[(~pd.isnull(dfmE[dfmE<0.5])).any(axis=1)].index
for cell_Id in cell_ID_small:
    a=df[df['cell_ID'] == cell_Id]['date']

exlude data from 1.Nov

In [ ]:
import datetime
def isdate(x):
    x=x[:10]
    #print(x)
    dat=datetime.datetime.strptime(x,'%a %b %d')
    #('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p'
    if dat.day == 1 and dat.month == 11:
        return False
    else:
        return True
    
   
df_red=df[df['date'].apply(isdate)]

exclude CellID 15- bad force curves

In [ ]:
df_red=df_red[df_red['cell_ID']!=15]

In [ ]:
df=df_red # 


df['varE^-1'] = 1 / df['var_E']

df['weighted_E'] = df['E'] * df['varE^-1']
df_mean_weightedE = df.groupby(['cell_Size','cell_ID'])['weighted_E'].mean()
df_mean_varInv = df.groupby(['cell_Size','cell_ID'])['varE^-1'].mean()
df_mean_E = df.groupby(['cell_Size','cell_ID'])['E'].mean()*1e-6
s=df.groupby(['cell_Size','cell_ID'])['var_E'].sum()
df_wE = df_mean_weightedE / df_mean_varInv * 1e-6


In [ ]:
#print(df_wE)
#print(df_mean_E)
dfwE=df_wE.unstack(level=0)
dfmE=df_mean_E.unstack(level=0)
#print(df3)
plt.figure(figsize=(20,20))
#plt.subplot(2,1,1)
p1=pd.melt(dfwE).plot(kind='scatter',x='cell_Size', y='value')
#p1=dfwE.plot(kind='box')


plt.title('weighted mean ')
#plt.subplot(2,1,2)
p2=pd.melt(dfmE).plot(kind='scatter',x='cell_Size', y='value')
#p2=dfmE.plot(kind='box')
plt.title('mean')
plt.Axes.set_ylabel(p2,'Young\'s modulus, MPa')
plt.Axes.set_ylabel(p1,'Young\'s modulus, MPa')
plt.Axes.set_ylim(p2,(0,6))
plt.Axes.set_ylim(p1,(0,6))


In [ ]:
print(len(dfwE[~pd.isnull(dfwE[5])][5]))
print(len(dfwE[~pd.isnull(dfwE[3])][3]))



In [ ]:
dfwE.describe()


In [ ]:
dfmE.describe()

t-test between cells with 3 µm and 5µm diameter

In [ ]:
from scipy.stats import ttest_ind
df_test=df_mean_E.unstack(level=0)

siz3=df_test[3]
siz5=df_test[5]

ttest_ind(siz3[~pd.isnull(siz3)], siz5[~pd.isnull(siz5)])


